This notebook performs data preprocessing for Big Mart sales dataset, which can be downloaded from Kaggle at [www.kaggle.com/devashish0507/big-mart-sales-prediction]

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("dataset/sales/Train.csv")
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [2]:
print(data.shape)
print("*"*10)
print(data .columns)

(8523, 12)
**********
Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')


In [3]:
# check the columns with null values
data.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [4]:
print(data['Item_Weight'].dtype)
print(data['Outlet_Size'].dtype)

float64
object


In [5]:
data.groupby(['Item_Type']).mean()['Item_Weight']

Item_Type
Baking Goods             12.277108
Breads                   11.346936
Breakfast                12.768202
Canned                   12.305705
Dairy                    13.426069
Frozen Foods             12.867061
Fruits and Vegetables    13.224769
Hard Drinks              11.400328
Health and Hygiene       13.142314
Household                13.384736
Meat                     12.817344
Others                   13.853285
Seafood                  12.552843
Snack Foods              12.987880
Soft Drinks              11.847460
Starchy Foods            13.690731
Name: Item_Weight, dtype: float64

In [6]:
# impute all the null values of Item_Weight using the mean, respective of the Item_Type
for i in data.Item_Type.value_counts().index:
    data.loc[(data['Item_Weight'].isna()) & (data['Item_Type'] == i),['Item_Weight']] = \
    data.loc[data['Item_Type'] == 'Fruits and Vegetables', ['Item_Weight']].mean()[0]

In [7]:
# split the data into numerical and categorical data frames and then impute the null values
cat_data = data.select_dtypes(object)
num_data = data.select_dtypes(np.number)
cat_data.isna().sum()

Item_Identifier            0
Item_Fat_Content           0
Item_Type                  0
Outlet_Identifier          0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
dtype: int64

In [8]:
cat_data.Outlet_Size.value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [9]:
cat_data.groupby(['Outlet_Type','Outlet_Size']).count()

Item_Identifier  Item_Fat_Content  Item_Type  \
Outlet_Type       Outlet_Size                                                 
Grocery Store     Small                    528               528        528   
Supermarket Type1 High                     932               932        932   
                  Medium                   930               930        930   
                  Small                   1860              1860       1860   
Supermarket Type2 Medium                   928               928        928   
Supermarket Type3 Medium                   935               935        935   

                               Outlet_Identifier  Outlet_Location_Type  
Outlet_Type       Outlet_Size                                           
Grocery Store     Small                      528                   528  
Supermarket Type1 High                       932                   932  
                  Medium                     930                   930  
                  Small                     1860                  1860  
Supermarket Type2 Medium                     928                   928  
Supermarket Type3 Medium                     935                   935

In [10]:
cat_data.loc[(cat_data['Outlet_Size'].isna()) & (cat_data['Outlet_Type'] == 'Grocery Store'), ['Outlet_Size']] = 'Small'
cat_data.loc[(cat_data['Outlet_Size'].isna()) & (cat_data['Outlet_Type'] == 'Supermarket Type1'), ['Outlet_Size']] = 'Small'
cat_data.loc[(cat_data['Outlet_Size'].isna()) & (cat_data['Outlet_Type'] == 'Supermarket Type2'), ['Outlet_Size']] = 'Medium'
cat_data.loc[(cat_data['Outlet_Size'].isna()) & (cat_data['Outlet_Type'] == 'Supermarket Type3'), ['Outlet_Size']] = 'Medium'

In [11]:
cat_data.Item_Fat_Content.value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [12]:
cat_data.loc[cat_data['Item_Fat_Content'] == 'LF' , ['Item_Fat_Content']] = 'Low Fat'
cat_data.loc[cat_data['Item_Fat_Content'] == 'reg' , ['Item_Fat_Content']] = 'Regular'
cat_data.loc[cat_data['Item_Fat_Content'] == 'low fat' , ['Item_Fat_Content']] = 'Low Fat'
cat_data.head()

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1
1,DRC01,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2
2,FDN15,Low Fat,Meat,OUT049,Medium,Tier 1,Supermarket Type1
3,FDX07,Regular,Fruits and Vegetables,OUT010,Small,Tier 3,Grocery Store
4,NCD19,Low Fat,Household,OUT013,High,Tier 3,Supermarket Type1


In [13]:
# encoding categorical features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_data = cat_data.apply(le.fit_transform)

In [14]:
# standardize categorical and numerical features
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
num_data = pd.DataFrame(ss.fit_transform(num_data.drop(['Item_Outlet_Sales'], axis=1)), columns = num_data.drop(['Item_Outlet_Sales'],axis=1).columns)
cat_data = pd.DataFrame(ss.fit_transform(cat_data.drop(['Item_Identifier'], axis=1)), columns = cat_data.drop(['Item_Identifier'], axis=1).columns)
final_data = pd.concat([num_data,cat_data],axis=1)
final_data.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,-0.856325,-0.970732,1.747454,0.139541,-0.738147,-0.766479,1.507813,-0.664080,-1.369334,-0.252658
1,-1.655730,-0.908111,-1.489023,1.334103,1.354743,1.608963,-0.607071,-0.664080,1.091569,1.002972
2,1.083061,-0.956917,0.010040,0.139541,-0.738147,0.658786,1.507813,-0.664080,-1.369334,-0.252658
3,1.485129,-1.281758,0.660050,0.020085,1.354743,-0.291391,-1.664513,0.799954,1.091569,-1.508289
4,-0.943834,-1.281758,-1.399220,-1.293934,-0.738147,0.421242,-1.312032,-2.128115,1.091569,-0.252658


In [15]:
X = final_data
y = data['Item_Outlet_Sales']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=5)